In [3]:
import os
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

face_model = cv2.dnn.readNetFromTorch("models/openface.nn4.small2.v1.t7")
face_detector = cv2.dnn.readNetFromCaffe("models/deploy.prototxt", "models/res10_300x300_ssd_iter_140000_fp16.caffemodel")

dataset_path = "known_faces"
X, Y = [], []

for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_folder):
        continue
    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue
        h, w = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        face_detector.setInput(blob)
        detections = face_detector.forward()
        if detections.shape[2] == 0:
            continue
        confidence = detections[0, 0, 0, 2]
        if confidence < 0.5:
            continue
        box = detections[0, 0, 0, 3:7] * np.array([w, h, w, h])
        (x1, y1, x2, y2) = box.astype("int")
        face = image[y1:y2, x1:x2]
        if face.size == 0:
            continue
        face_blob = cv2.dnn.blobFromImage(cv2.resize(face, (96, 96)), 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        face_model.setInput(face_blob)
        vec = face_model.forward()
        X.append(vec.flatten())
        Y.append(person_name)

le = LabelEncoder()
labels = le.fit_transform(Y)
clf = SVC(C=1.0, kernel="linear", probability=True)
clf.fit(X, labels)

with open("classifier.pkl", "wb") as f:
    pickle.dump((le, clf), f)